<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250702/%E6%9C%80%E5%A4%A7%E6%B5%81%E5%95%8F%E9%A1%8C%E3%81%AE%E6%BC%94%E7%BF%92%E5%95%8F%E9%A1%8C%E3%81%AE%E8%A7%A3%E8%AA%AC%E3%81%A8%E8%A7%A3%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#最大流問題

## はじめに
演習問題の解答に必要なライブラリをインストールする。

In [1]:
%%capture
!pip install mip

---
## 演習問題1

### 1. 問題
例題のアーク (A,T)の容量を5から7に増やし、アーク (S,B)の容量を8から6に減らした場合、最大フロー量を求めよ。

### 2. 数理モデルの定式化
この問題は、例題のネットワーク構造を基本とし、一部のアークの容量のみを変更した最大流問題である。基本的な数理モデルの構造は例題と同じである。

* **集合とパラメータ:**
  * ノード集合 $N = \{S, A, B, T\}$
  * 始点（ソース）$s = S$
  * 終点（シンク）$t = T$
  * アークとその容量 $u_{ij}$:
    * $u_{SA} = 10$
    * $u_{SB} = 6$  **(変更点)**
    * $u_{AB} = 3$
    * $u_{AT} = 7$  **(変更点)**
    * $u_{BT} = 12$
  * アークの集合 $A = \{(S,A), (S,B), (A,B), (A,T), (B,T)\}$

* **決定変数:**
    * $x_{ij}$: アーク $(i,j) \in A$ 上を流れるフローの量。
    * $v_{total}$: ソースからシンクへの総フロー量。

* **目的関数:**
    ソースから流れ出す総フロー量 $v_{total}$ を最大化する。

$$\text{Maximize} \quad v_{total}$$

* **制約条件:**
  1.  **フロー保存則:**
   * ソース$S$:
$$\sum_{j | (S,j) \in A} x_{Sj} - \sum_{i | (i,S) \in A} x_{iS} = v_{total}$$
   * シンク $T$:
$$\sum_{i | (i,T) \in A} x_{iT} - \sum_{j | (T,j) \in A} x_{Tj} = v_{total}$$
   * 中間ノード $k \in N \setminus \{s, t\}$:
$$\sum_{j | (k,j) \in A} x_{kj} - \sum_{i | (i,k) \in A} x_{ik} = 0$$
  2.  **容量制約:**
        全てのアーク $(i,j) \in A$ において、$0 \le x_{ij} \le u_{ij}$ が成立する。

### 3. Python (MIP) による実装


In [2]:
import pandas as pd
from mip import Model, xsum, maximize, CONTINUOUS, OptimizationStatus

# 問題データ定義
nodes = ['S', 'A', 'B', 'T']
source_node = 'S'
sink_node = 'T'

# アークの定義: (始点, 終点, 容量)
# (S,B)の容量を8->6に、(A,T)の容量を5->7に変更
arcs_data = [
    ('S', 'A', 10),
    ('S', 'B', 6),   # 容量変更
    ('A', 'B', 3),
    ('A', 'T', 7),   # 容量変更
    ('B', 'T', 12)
]

# アークとその容量を辞書として格納
defined_arcs = [(u, v) for u, v, cap in arcs_data]
capacities = {(u, v): cap for u, v, cap in arcs_data}

# モデルの作成
model = Model("MaxFlow_Ex1")

# 決定変数: x_flow[i,j] はアーク(i,j)上のフロー量
x_flow = {(i, j): model.add_var(name=f"x_{i}_{j}", lb=0, ub=capacities[i, j]) for (i, j) in defined_arcs}

# 決定変数: v_total_flow は総フロー量
v_total_flow = model.add_var(name="v_total")

# 目的関数: 総フロー量 v_total_flow の最大化
model.objective = maximize(v_total_flow)

# 制約条件: フロー保存則
for k in nodes:
    # ノードkから出ていくフローの総和
    flow_out = xsum(x_flow[i, j] for (i, j) in defined_arcs if i == k)
    # ノードkに入ってくるフローの総和
    flow_in = xsum(x_flow[i, j] for (i, j) in defined_arcs if j == k)

    if k == source_node:
        # ソース: (出ていくフロー) - (入ってくるフロー) = v_total
        model += flow_out - flow_in == v_total_flow, f"FlowBalance_{k}"
    elif k == sink_node:
        # シンク: (入ってくるフロー) - (出ていくフロー) = v_total
        model += flow_in - flow_out == v_total_flow, f"FlowBalance_{k}"
    else:
        # 中間ノード: (出ていくフロー) = (入ってくるフロー)
        model += flow_out - flow_in == 0, f"FlowBalance_{k}"

# モデルの最適化
status = model.optimize()

# --- 結果の表示 ---
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最大フロー量: {model.objective_value:.2f}")
    print("\n各アークのフロー量:")
    for (i, j) in defined_arcs:
        if x_flow[i, j].x > 1e-6: # ごく小さいフローは無視
            print(f"  アーク ({i} -> {j}): {x_flow[i, j].x:.2f} (容量: {capacities[i, j]})")
elif status == OptimizationStatus.INFEASIBLE:
    print("実行不可能な問題です。制約を満たす解が存在しません。")
else:
    print(f"最適化が停止しました。ステータス: {status}")

最適解が見つかりました。
最大フロー量: 16.00

各アークのフロー量:
  アーク (S -> A): 10.00 (容量: 10)
  アーク (S -> B): 6.00 (容量: 6)
  アーク (A -> B): 3.00 (容量: 3)
  アーク (A -> T): 7.00 (容量: 7)
  アーク (B -> T): 9.00 (容量: 12)


---
## 演習問題2

### 1. 問題
例題のネットワークに、新たにノードBからノードAへのアーク (B, A) を追加することを考える。この新しいアークの容量は2であるとする。この時の最大フロー量を求めよ。

### 2. 数理モデルの定式化
例題のネットワークに新しいアーク `(B,A)` を追加するため、アークの集合と容量の定義が変更される。

* **集合とパラメータ:**
  * ノード集合 $N = \{S, A, B, T\}$
  * 始点（ソース）$s = S$
  * 終点（シンク）$t = T$
  * アークとその容量 $u_{ij}$:
    * $u_{SA} = 10$, $u_{SB} = 8$, $u_{AB} = 3$, $u_{AT} = 5$, $u_{BT} = 12$
    * $u_{BA} = 2$ **(追加アーク)**
  * アークの集合 $A = \{(S,A), (S,B), (A,B), (A,T), (B,T), (B,A)\}$ **(変更点)**

その他の決定変数、目的関数、制約条件の定義は演習問題1と同様である。

### 3. Python (MIP) による実装

In [3]:
import pandas as pd
from mip import Model, xsum, maximize, CONTINUOUS, OptimizationStatus

# 問題データ定義
nodes = ['S', 'A', 'B', 'T']
source_node = 'S'
sink_node = 'T'

# アークの定義: (始点, 終点, 容量)
# 例題に (B,A) 容量2 を追加
arcs_data = [
    ('S', 'A', 10),
    ('S', 'B', 8),
    ('A', 'B', 3),
    ('A', 'T', 5),
    ('B', 'T', 12),
    ('B', 'A', 2)    # 追加アーク
]

# アークとその容量を辞書として格納
defined_arcs = [(u, v) for u, v, cap in arcs_data]
capacities = {(u, v): cap for u, v, cap in arcs_data}

# モデルの作成
model = Model("MaxFlow_Ex2")

# 決定変数: x_flow[i,j] はアーク(i,j)上のフロー量
x_flow = {(i, j): model.add_var(name=f"x_{i}_{j}", lb=0, ub=capacities[i, j]) for (i, j) in defined_arcs}

# 決定変数: v_total_flow は総フロー量
v_total_flow = model.add_var(name="v_total")

# 目的関数: 総フロー量 v_total_flow の最大化
model.objective = maximize(v_total_flow)

# 制約条件: フロー保存則
for k in nodes:
    flow_out = xsum(x_flow[i, j] for (i, j) in defined_arcs if i == k)
    flow_in = xsum(x_flow[i, j] for (i, j) in defined_arcs if j == k)

    if k == source_node:
        model += flow_out - flow_in == v_total_flow, f"FlowBalance_{k}"
    elif k == sink_node:
        model += flow_in - flow_out == v_total_flow, f"FlowBalance_{k}"
    else:
        model += flow_out - flow_in == 0, f"FlowBalance_{k}"

# モデルの最適化
status = model.optimize()

# --- 結果の表示 ---
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最大フロー量: {model.objective_value:.2f}")
    print("\n各アークのフロー量:")
    for (i, j) in defined_arcs:
        if x_flow[i, j].x > 1e-6: # ごく小さいフローは無視
            print(f"  アーク ({i} -> {j}): {x_flow[i, j].x:.2f} (容量: {capacities[i, j]})")
elif status == OptimizationStatus.INFEASIBLE:
    print("実行不可能な問題です。制約を満たす解が存在しません。")
else:
    print(f"最適化が停止しました。ステータス: {status}")

最適解が見つかりました。
最大フロー量: 16.00

各アークのフロー量:
  アーク (S -> A): 8.00 (容量: 10)
  アーク (S -> B): 8.00 (容量: 8)
  アーク (A -> B): 3.00 (容量: 3)
  アーク (A -> T): 5.00 (容量: 5)
  アーク (B -> T): 11.00 (容量: 12)


---
## 演習問題3

### 1. 問題
例題のネットワークにおいて、アーク (A,B)が使用できなくなったとする。この時、最大フローを求めよ。

### 2. 数理モデルの定式化
アーク `(A,B)` をネットワークから除去するため、アークの集合の定義が変更される。

* **集合とパラメータ:**
  * ノード集合 $N = \{S, A, B, T\}$
  * 始点（ソース）$s = S$
  * 終点（シンク）$t = T$
  * アークとその容量 $u_{ij}$:
   * $u_{SA} = 10$, $u_{SB} = 8$, $u_{AT} = 5$, $u_{BT} = 12$
  * アークの集合 $A = \{(S,A), (S,B), (A,T), (B,T)\}$ **(変更点)**

その他の決定変数、目的関数、制約条件の定義は演習問題1と同様である。

### 3. Python (MIP) による実装
以下に、アーク `(A,B)` を除外したモデルを `python-mip` で実装したコードを示す。

In [4]:
import pandas as pd
from mip import Model, xsum, maximize, CONTINUOUS, OptimizationStatus

# 問題データ定義
nodes = ['S', 'A', 'B', 'T']
source_node = 'S'
sink_node = 'T'

# アークの定義: (始点, 終点, 容量)
# (A,B) を除去
arcs_data = [
    ('S', 'A', 10),
    ('S', 'B', 8),
    # ('A', 'B', 3), # 除去
    ('A', 'T', 5),
    ('B', 'T', 12)
]

# アークとその容量を辞書として格納
defined_arcs = [(u, v) for u, v, cap in arcs_data]
capacities = {(u, v): cap for u, v, cap in arcs_data}

# モデルの作成
model = Model("MaxFlow_Ex3")

# 決定変数: x_flow[i,j] はアーク(i,j)上のフロー量
x_flow = {(i, j): model.add_var(name=f"x_{i}_{j}", lb=0, ub=capacities[i, j]) for (i, j) in defined_arcs}

# 決定変数: v_total_flow は総フロー量
v_total_flow = model.add_var(name="v_total")

# 目的関数: 総フロー量 v_total_flow の最大化
model.objective = maximize(v_total_flow)

# 制約条件: フロー保存則
for k in nodes:
    flow_out = xsum(x_flow[i, j] for (i, j) in defined_arcs if i == k)
    flow_in = xsum(x_flow[i, j] for (i, j) in defined_arcs if j == k)

    if k == source_node:
        model += flow_out - flow_in == v_total_flow, f"FlowBalance_{k}"
    elif k == sink_node:
        model += flow_in - flow_out == v_total_flow, f"FlowBalance_{k}"
    else:
        model += flow_out - flow_in == 0, f"FlowBalance_{k}"

# モデルの最適化
status = model.optimize()

# --- 結果の表示 ---
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最大フロー量: {model.objective_value:.2f}")
    print("\n各アークのフロー量:")
    for (i, j) in defined_arcs:
        if x_flow[i, j].x > 1e-6: # ごく小さいフローは無視
            print(f"  アーク ({i} -> {j}): {x_flow[i, j].x:.2f} (容量: {capacities[i, j]})")
elif status == OptimizationStatus.INFEASIBLE:
    print("実行不可能な問題です。制約を満たす解が存在しません。")
else:
    print(f"最適化が停止しました。ステータス: {status}")

最適解が見つかりました。
最大フロー量: 13.00

各アークのフロー量:
  アーク (S -> A): 5.00 (容量: 10)
  アーク (S -> B): 8.00 (容量: 8)
  アーク (A -> T): 5.00 (容量: 5)
  アーク (B -> T): 8.00 (容量: 12)


---
## 演習問題4

### 1. 問題の確認
例題のネットワークでは、アーク (S,B) 容量8、(A,B) 容量3、(A,T) 容量5が飽和していました。

1.  飽和していないアーク (S,A) (フロー8、容量10)の容量を10から12に増やした場合、最大フロー量を求めよ。
2.  次に、元の容量に戻した後、飽和しているアーク (A,T) の容量を5から7に増やした場合、最大フロー量を求めよ。

---
### 4-1. 飽和していないアークの容量増加

#### **a. 数理モデルの定式化**
アーク `(S,A)` の容量を10から12に変更する。

* **集合とパラメータ:**
  * アークとその容量 $u_{ij}$:
    * $u_{SA} = 12$ **(変更点)**
    * $u_{SB} = 8$, $u_{AB} = 3$, $u_{AT} = 5$, $u_{BT} = 12$

#### **b. Python (MIP) による実装**

In [5]:
import pandas as pd
from mip import Model, xsum, maximize, CONTINUOUS, OptimizationStatus

# 問題データ定義
nodes = ['S', 'A', 'B', 'T']
source_node = 'S'
sink_node = 'T'

# アークの定義: (始点, 終点, 容量)
# (S,A)の容量を10->12に変更
arcs_data = [
    ('S', 'A', 12),  # 容量変更
    ('S', 'B', 8),
    ('A', 'B', 3),
    ('A', 'T', 5),
    ('B', 'T', 12)
]

# アークとその容量を辞書として格納
defined_arcs = [(u, v) for u, v, cap in arcs_data]
capacities = {(u, v): cap for u, v, cap in arcs_data}

# モデルの作成
model = Model("MaxFlow_Ex4-1")

# 変数、目的関数、制約の定義（定型的なため詳細は省略）
x_flow = {(i, j): model.add_var(name=f"x_{i}_{j}", lb=0, ub=capacities[i, j]) for (i, j) in defined_arcs}
v_total_flow = model.add_var(name="v_total")
model.objective = maximize(v_total_flow)
for k in nodes:
    flow_out = xsum(x_flow[i, j] for (i, j) in defined_arcs if i == k)
    flow_in = xsum(x_flow[i, j] for (i, j) in defined_arcs if j == k)
    if k == source_node: model += flow_out - flow_in == v_total_flow
    elif k == sink_node: model += flow_in - flow_out == v_total_flow
    else: model += flow_out - flow_in == 0

# モデルの最適化と結果表示
status = model.optimize()
if status == OptimizationStatus.OPTIMAL:
    print(f"最大フロー量: {model.objective_value:.2f}")
else:
    print(f"最適化失敗: {status}")

最大フロー量: 16.00


---
### 4-2. 飽和しているアークの容量増加

#### **a. 数理モデルの定式化**
ネットワークを元の状態に戻し、アーク `(A,T)` の容量を5から7に変更する。

* **集合とパラメータ:**
  * アークとその容量 $u_{ij}$:
    * $u_{AT} = 7$ **(変更点)**
    * $u_{SA} = 10$, $u_{SB} = 8$, $u_{AB} = 3$, $u_{BT} = 12$

#### **b. Python (MIP) による実装**

In [6]:
import pandas as pd
from mip import Model, xsum, maximize, CONTINUOUS, OptimizationStatus

# 問題データ定義
nodes = ['S', 'A', 'B', 'T']
source_node = 'S'
sink_node = 'T'

# アークの定義: (始点, 終点, 容量)
# (A,T)の容量を5->7に変更
arcs_data = [
    ('S', 'A', 10),
    ('S', 'B', 8),
    ('A', 'B', 3),
    ('A', 'T', 7),   # 容量変更
    ('B', 'T', 12)
]

# アークとその容量を辞書として格納
defined_arcs = [(u, v) for u, v, cap in arcs_data]
capacities = {(u, v): cap for u, v, cap in arcs_data}

# モデルの作成
model = Model("MaxFlow_Ex4-2")

# 変数、目的関数、制約の定義（定型的なため詳細は省略）
x_flow = {(i, j): model.add_var(name=f"x_{i}_{j}", lb=0, ub=capacities[i, j]) for (i, j) in defined_arcs}
v_total_flow = model.add_var(name="v_total")
model.objective = maximize(v_total_flow)
for k in nodes:
    flow_out = xsum(x_flow[i, j] for (i, j) in defined_arcs if i == k)
    flow_in = xsum(x_flow[i, j] for (i, j) in defined_arcs if j == k)
    if k == source_node: model += flow_out - flow_in == v_total_flow
    elif k == sink_node: model += flow_in - flow_out == v_total_flow
    else: model += flow_out - flow_in == 0

# モデルの最適化と結果表示
status = model.optimize()
if status == OptimizationStatus.OPTIMAL:
    print(f"最大フロー量: {model.objective_value:.2f}")
else:
    print(f"最適化失敗: {status}")

最大フロー量: 18.00


---
## 演習問題5

### 1. 問題の確認
以下の複数ソース・複数シンクを持つネットワークの最大フロー量を求めよ。

* ソース: S1, S2
* シンク: T1, T2
* 中間ノード: M
* アークと容量:
    * (S1, M, 10), (S2, M, 8)
    * (M, T1, 7), (M, T2, 9)

### 2. 数理モデルの定式化
この問題は複数ソース・複数シンクを持つため、標準的な単一ソース・単一シンクの最大流問題に変換する必要がある。そのために、**スーパーソース** と **スーパーシンク** という仮想的なノードを導入する。

1.  **スーパーソースの導入:**
    * 仮想的な始点（スーパーソース） `S_super` を追加する。
    * `S_super` から全ての実際のソース（S1, S2）へアークを追加する。これらのアークの容量は、元のソースから出ていくフローを制限しないように、十分に大きな値（または無限大）に設定する。

2.  **スーパーシンクの導入:**
    * 仮想的な終点（スーパーシンク） `T_super` を追加する。
    * 全ての実際のシンク（T1, T2）から `T_super` へアークを追加する。これらのアークの容量も同様に、十分に大きな値に設定する。

この変換により、`S_super` から `T_super` への単一始点・単一終点の最大流問題として定式化できる。

* **集合とパラメータ:**
  * ノード集合 $N = \{S_{super}, S1, S2, M, T1, T2, T_{super}\}$
  * 始点（ソース）$s = S_{super}$
  * 終点（シンク）$t = T_{super}$
  * アークの集合 $A$ とその容量 $u_{ij}$:
    * 元のネットワーク: $u_{S1,M}=10, u_{S2,M}=8, u_{M,T1}=7, u_{M,T2}=9$
    * 追加アーク: $u_{S_{super},S1}=\infty, u_{S_{super},S2}=\infty, u_{T1,T_{super}}=\infty, u_{T2,T_{super}}=\infty$
        （実装上は、ネットワーク内の全実アーク容量の合計など、十分に大きい有限値を用いる）

決定変数、目的関数、制約条件の型は、これまでの問題と同様である。

### 3. Python (MIP) による実装
以下に、スーパーソースとスーパーシンクを導入したモデルのコードを示す。

In [7]:
import pandas as pd
from mip import Model, xsum, maximize, CONTINUOUS, OptimizationStatus

# 問題データ定義
# 元のノードにスーパーソースとスーパーシンクを追加
nodes = ['S_super', 'S1', 'S2', 'M', 'T1', 'T2', 'T_super']
source_node = 'S_super'
sink_node = 'T_super'

# アークの定義: (始点, 終点, 容量)
# 元のアーク
original_arcs_data = [
    ('S1', 'M', 10),
    ('S2', 'M', 8),
    ('M', 'T1', 7),
    ('M', 'T2', 9)
]

# スーパーソース/シンクとの接続アーク
# 容量は十分に大きい値とする（ここでは元の容量の合計）
large_capacity = sum(cap for u, v, cap in original_arcs_data)

super_arcs_data = [
    ('S_super', 'S1', large_capacity),
    ('S_super', 'S2', large_capacity),
    ('T1', 'T_super', large_capacity),
    ('T2', 'T_super', large_capacity)
]

arcs_data = original_arcs_data + super_arcs_data

# アークとその容量を辞書として格納
defined_arcs = [(u, v) for u, v, cap in arcs_data]
capacities = {(u, v): cap for u, v, cap in arcs_data}

# モデルの作成
model = Model("MaxFlow_Ex5")

# 決定変数: x_flow[i,j] はアーク(i,j)上のフロー量
x_flow = {(i, j): model.add_var(name=f"x_{i}_{j}", lb=0, ub=capacities[i, j]) for (i, j) in defined_arcs}

# 決定変数: v_total_flow は総フロー量
v_total_flow = model.add_var(name="v_total")

# 目的関数: 総フロー量 v_total_flow の最大化
model.objective = maximize(v_total_flow)

# 制約条件: フロー保存則
for k in nodes:
    flow_out = xsum(x_flow[i, j] for (i, j) in defined_arcs if i == k)
    flow_in = xsum(x_flow[i, j] for (i, j) in defined_arcs if j == k)

    if k == source_node:
        model += flow_out - flow_in == v_total_flow, f"FlowBalance_{k}"
    elif k == sink_node:
        model += flow_in - flow_out == v_total_flow, f"FlowBalance_{k}"
    else:
        model += flow_out - flow_in == 0, f"FlowBalance_{k}"

# モデルの最適化
status = model.optimize()

# --- 結果の表示 ---
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最大フロー量: {model.objective_value:.2f}")
    print("\n各アークのフロー量:")
    for (i, j) in defined_arcs:
        if x_flow[i, j].x > 1e-6: # ごく小さいフローは無視
            print(f"  アーク ({i} -> {j}): {x_flow[i, j].x:.2f} (容量: {capacities[i, j]})")
elif status == OptimizationStatus.INFEASIBLE:
    print("実行不可能な問題です。制約を満たす解が存在しません。")
else:
    print(f"最適化が停止しました。ステータス: {status}")

最適解が見つかりました。
最大フロー量: 16.00

各アークのフロー量:
  アーク (S1 -> M): 10.00 (容量: 10)
  アーク (S2 -> M): 6.00 (容量: 8)
  アーク (M -> T1): 7.00 (容量: 7)
  アーク (M -> T2): 9.00 (容量: 9)
  アーク (S_super -> S1): 10.00 (容量: 34)
  アーク (S_super -> S2): 6.00 (容量: 34)
  アーク (T1 -> T_super): 7.00 (容量: 34)
  アーク (T2 -> T_super): 9.00 (容量: 34)
